# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

training_file = "./traffic-signs-data/preprocessed-train.p"
testing_file = "./traffic-signs-data/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [ ]:
### Replace each question mark with the appropriate value.

n_train = len(X_train)

n_test = len(X_test)

image_shape = X_train[0].shape

n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

In [ ]:
import numpy as np
from collections import Counter

In [ ]:
import csv

sign_names = []
with open('./signnames.csv', 'rt') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if row[0].isdigit():
            sign_names.append(row[1])

In [ ]:
x = np.arange(n_classes)
label_counter = Counter(y_train)
plt.title("Labels distribution in train.p")
plt.bar(x, height = label_counter.values())
plt.xticks(x, label_counter.keys());

In [ ]:
traffic_signs_images = []
for label in range(n_classes):
    indices = np.where(y_train == label)[0]
    traffic_signs_images.append(X_train[indices])
    
f = plt.figure(figsize=(n_classes,n_classes))
for label in range(n_classes):
    image = traffic_signs_images[label][0].squeeze()
    f.add_subplot(n_classes, 1, label + 1)
    plt.title(sign_names[label])
    plt.imshow(image)
    plt.tight_layout()
    plt.gca().axes.get_xaxis().set_visible(False)
    plt.gca().axes.get_yaxis().set_visible(False)
plt.show()

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Note: training data are preprocessed.

### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**

The samples in train.p is not in uniform distribution. Some labels have more then 2000 samples, some labels have only ~200 samples. If we perform training on these biased data, our network will be biased too. 

Let say there are only two labels in our dataset, and we have 900 samples for label _A_, 100 label for label _B_. If our network always claimed that all the training data are in label _A_, then it will reach 90% accuracy. However, in reality, our model knows nothing about label _B_, and it is not reliable at all.

In order to solve this issue. I generate new samples by duplicating given training data, such that all labels have the same number of samples.

Although we have uniform distributed training data, the same sample may exist in the dataset muliple times, which is not good too. The network trained with these repeated samples maybe overfitted.

For example, if we teach our network the meaning of a human with only one image, then our network may think that a football always look exact the same as the one it saw in the image, and any human being taller or shorter then the one in the image may not be classified as _human_.

We can solve this problem by adjusting sample images. Affine transformation is the one and only one way I choose. I transform all the sampe images I have randomly. There should be no  two images be the same, or the duplicated samples could not affect overall network accuracy.

There are many still many other way to adjust the images, such as changing the brightness of images, however, I didn't do so, because my network reach ~0.95 test accuracy without adjusting the brightness.

### The preprocessing step can be found in another notebook, `_Traffic_Sign_Classifier_(Preprocessing)`

In [ ]:
### split training data into training set and validation set, in ratio 4:1.
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:
### shuffle training set
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train, random_state=0)

In [ ]:
x = np.arange(n_classes)
label_counter = Counter(y_train)
plt.title("Labels distribution in training set")
plt.bar(x, height = label_counter.values())
plt.xticks(x, label_counter.keys());

In [ ]:
x = np.arange(n_classes)
label_counter = Counter(y_validation)
plt.title("Labels distribution in validation set")
plt.bar(x, height = label_counter.values())
plt.xticks(x, label_counter.keys());

In [ ]:
x = np.arange(n_classes)
label_counter = Counter(y_test)

for i in range(n_classes):
    if label_counter[i] == 0:
        label_counter[i] = 0

plt.title("Labels distribution in test set")
plt.bar(x, height = label_counter.values())
plt.xticks(x, label_counter.keys());

### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**

I created training set and validation set from preprocessed training data. 1/5 of training set are selected to be validation set. The selection is in random.

Test set are from the given test data. No processing are done on these test data

In [ ]:
import tensorflow as tf

EPOCHS = 150
BATCH_SIZE = 1024

In [ ]:
from tensorflow.contrib.layers import flatten

def LeNet(x, keep_prob):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x15.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 15), mean = mu, stddev = sigma),name="conv1_W")
    conv1_b = tf.Variable(tf.zeros(15),name="conv1_b")
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # Activation.
    conv1 = tf.nn.relu(conv1)

    # Pooling. Input = 28x28x15. Output = 14x14x15.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Layer 2: Convolutional. Input = 14x14x15. Output = 10x10x25.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 15, 25), mean = mu, stddev = sigma),name="conv2_W")
    conv2_b = tf.Variable(tf.zeros(25),name="conv2_b")
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # Activation.
    conv2 = tf.nn.relu(conv2)

    # Pooling. Input = 10x10x25. Output = 5x5x25.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Flatten. Input = 5x5x25. Output = 625.
    fc0   = flatten(conv2)
    
    # Dropout.
    fc0 = tf.nn.dropout(fc0, keep_prob, seed=0)

    # Layer 3: Fully Connected. Input = 625. Output = 150.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(625, 150), mean = mu, stddev = sigma),name="fc1_W")
    fc1_b = tf.Variable(tf.zeros(150),name="fc1_b")
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # Activation.
    fc1    = tf.nn.relu(fc1)
    
    # Dropout.
    fc1 = tf.nn.dropout(fc1, keep_prob, seed=0)

    # Layer 4: Fully Connected. Input = 150. Output = 100.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(150, 100), mean = mu, stddev = sigma),name="fc2_W")
    fc2_b  = tf.Variable(tf.zeros(100),name="fc2_b")
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # Activation.
    fc2    = tf.nn.relu(fc2)

    # Layer 5: Fully Connected. Input = 100. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(100, n_classes), mean = mu, stddev = sigma),name="fc3_W")
    fc3_b  = tf.Variable(tf.zeros(n_classes),name="fc3_b")
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

In [ ]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 43)

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

My network is based on the convolution neural network, LeNet, provided in LeNet Lab. Because traffic signs looks more complicated then the handwritings, so I increase the size of each layer.

My network has two convolution layers and three fully connected layers. The details of each layer can be found in below.

***************************************

Shape: 32x32x32_

### layer 1, convolutional
##### convolution
* filter shape: (5, 5, 3, 15), strides: [1, 1, 1, 1], with valid padding

#### relu

_Shape: 28x28x15_

#### max pool
* kernel: [1, 2, 2, 1], strides: [1, 2, 2, 1], with valid padding

_Shape: 14x14x15_

### layer 2, convolutional
##### convolution
* filter shape: (5, 5, 15, 25), strides: [1, 1, 1, 1], with valid padding

#### relu

_Shape: 28x28x15_

#### max pool
* kernel: [1, 2, 2, 1], strides: [1, 2, 2, 1], with valid padding

_Shape: 5x5x25_
    
#### flatten

_Node: 625_

#### drop out
* keep probability: 0.75(training) or 1.0(testing)

_Node: 625_

### Layer 3, fully connected
#### relu

#### drop out
* keep probability: 0.75(training) or 1.0(testing)

_Node: 150_

### Layer 4, fully connected
#### relu

#### drop out
* keep probability: 0.75(training) or 1.0(testing)

_Node: 100_

### Layer 5, fully connected

_Node: 43_

***************************************

Besides the difference in the layer size, I did add a dropout at the first two fully connected layer. Adding dropout helps reduce the rate of overfitting. I didn't add the dropout to the last fully connected layer, because the properties available on the layer is just 100. Both of them may be an important feature and dropping anyone of them may lead to model to underfitting.

In [ ]:
rate = 0.001

logits = LeNet(x, keep_prob)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()
# The file path to save the data
save_file = './model.ckpt'

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train, random_state=0)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 0.75})
            
        training_accuracy = evaluate(X_train, y_train)
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Training Accuracy = {:.3f}".format(training_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    # Save the model
    saver.save(sess, save_file)

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, save_file)

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

I have increased the batch size of the network. Because we have a powerful machine, which can handle more then 200 sample in a batch, for training, so we can use a higher batch number.

On the other hand, each epoch can be finished in a shorter time, we can increse the number of epochs too.

AdamOptimizer used in LeNet looks working well in our network, so I didn't change it.

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

Wothout changing any parameter of LeNet, the accuracy is ~0.9

At the begining, I did muliple the size of each layer by 10, and I found out that the accuracy is lower then without modifying any parameter in LeNet. I though my network is overfitting, so I tried use a smaller learning rate, but no help.

Then I try use a smaller muliplier, and the accuracy became better and better. Finally it reach training accuracy ~0.97 and test accuracy ~0.92.

The diff between them should be related to overfitting, so I applied dropout.

First I tried adding drop out to all fully connected layer. The accuracy suddenly dropped to 0.02, which means my network is almost use guessing to determine the label.

I noticed that I may drop too much from the layer, so I reduce the number of dropout layer and increse the keep properties.

Finally my network reaches ~0.99 training accuracy and ~0.95 test accuracy.

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

## Please clear the kenerl before executing the following cells, if there is any error occurs.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

In [ ]:
# Load pickled data
import pickle

testing_file = "./traffic-signs-data/custom-test.p"

with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_test, y_test = test['features'], test['labels']


In [ ]:
n_test = len(X_test)

n_classes = 43

In [ ]:
import csv

sign_names = []
with open('./signnames.csv', 'rt') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if row[0].isdigit():
            sign_names.append(row[1])

In [ ]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

In [ ]:
y_name = []
for i in range(n_test):
    y_name.append(sign_names[y_test[i]])
    
f = plt.figure(figsize=(len(X_test),len(X_test)))
for i in range(len(X_test)):
        image = X_test[i].squeeze()
        f.add_subplot(len(X_test), 1, i + 1)
        plt.title(sign_names[y_test[i]])
        plt.imshow(image)
        plt.tight_layout()
        plt.gca().axes.get_xaxis().set_visible(False)
        plt.gca().axes.get_yaxis().set_visible(False)
plt.show()

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**

These images are traffic signs I found in Hong Kong. These signs look similar to those provided in the given training set. These images should not be unknown to my network.

By the way, some of the images, the 1st, 4th and 5th, images in the dataset are in poor light condition. However, there are images in poor light condiftion can be found in the given training set. That should should not be a problem for my network too.

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

In [ ]:
import tensorflow as tf

BATCH_SIZE = 1024

In [ ]:
from tensorflow.contrib.layers import flatten

def LeNet(x, keep_prob):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x15.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 15), mean = mu, stddev = sigma),name="conv1_W")
    conv1_b = tf.Variable(tf.zeros(15),name="conv1_b")
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # Activation.
    conv1 = tf.nn.relu(conv1)

    # Pooling. Input = 28x28x15. Output = 14x14x15.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Layer 2: Convolutional. Input = 14x14x25. Output = 10x10x25.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 15, 25), mean = mu, stddev = sigma),name="conv2_W")
    conv2_b = tf.Variable(tf.zeros(25),name="conv2_b")
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # Activation.
    conv2 = tf.nn.relu(conv2)

    # Pooling. Input = 10x10x25. Output = 5x5x25.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # Flatten. Input = 5x5x25. Output = 625.
    fc0   = flatten(conv2)
    
    # Dropout.
    fc0 = tf.nn.dropout(fc0, keep_prob, seed=0)

    # Layer 3: Fully Connected. Input = 625. Output = 150.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(625, 150), mean = mu, stddev = sigma),name="fc1_W")
    fc1_b = tf.Variable(tf.zeros(150),name="fc1_b")
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # Activation.
    fc1    = tf.nn.relu(fc1)
    
    # Dropout.
    fc1 = tf.nn.dropout(fc1, keep_prob, seed=0)

    # Layer 4: Fully Connected. Input = 150. Output = 100.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(150, 100), mean = mu, stddev = sigma),name="fc2_W")
    fc2_b  = tf.Variable(tf.zeros(100),name="fc2_b")
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # Activation.
    fc2    = tf.nn.relu(fc2)

    # Layer 5: Fully Connected. Input = 100. Output = 43.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(100, n_classes), mean = mu, stddev = sigma),name="fc3_W")
    fc3_b  = tf.Variable(tf.zeros(n_classes),name="fc3_b")
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

In [ ]:
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
keep_prob = tf.placeholder(tf.float32)
one_hot_y = tf.one_hot(y, 43)

In [ ]:
logits = LeNet(x, keep_prob)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()
# The file path to save the data
save_file = './model.ckpt'

def evaluate(X_data, y_data):
    sess = tf.get_default_session()
    accuracy = sess.run(accuracy_operation, feed_dict={x: [X_data], y: [y_data], keep_prob: 1.0})
    return accuracy

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, save_file)
    
    over_all_acuracy = 0
    f = plt.figure(figsize=(len(X_test),len(X_test)))
    for i in range(len(X_test)):
        image = X_test[i].squeeze()
        test_accuracy = evaluate(X_test[i], y_test[i])   
        f.add_subplot(len(X_test), 1, i + 1)
        if test_accuracy == 1:
            over_all_acuracy = over_all_acuracy + 1/len(X_test)
            plt.title("Pass: " + sign_names[y_test[i]])
        else:
            plt.title("Fail: " + sign_names[y_test[i]])
        plt.imshow(image)
        plt.tight_layout()
        plt.gca().axes.get_xaxis().set_visible(False)
        plt.gca().axes.get_yaxis().set_visible(False)
    plt.show()
    print("Test Accuracy = {:.3f}".format(over_all_acuracy))

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

My network can identifiy the first three images I provoided. However, it failed to detect the sign in the last two images. So the over all accuracy of my network on these new image is only 60%.

However, the test accuracy of the original test set is as high as 96%.

The different between two value can be related to the new test data size or the overfitting

Because there are 5 images in new test set, so any misclassification may lead to a huge drop in accuracy. 

On the other hand, overfitting may cause huge difference between the value.

If there is no overfitting, then the posibilty that two images be misclassified is 0.05*0.05 = 0.0025, which is almost impossible, so I believed my network is overfitting

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

In [ ]:
top_k_prediction = tf.nn.top_k(tf.nn.softmax(logits), n_classes) 

with tf.Session() as sess:
    saver.restore(sess, save_file)
    
    values, indices = sess.run(top_k_prediction, feed_dict={x: X_test, keep_prob: 1.0})

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

prediction_matrix = []
for i in range(n_test):
    predictions = []
    for label in range(n_classes):
        predictions.append(values[i][np.where(indices[i]==label)[0][0]])
    prediction_matrix.append(predictions)
    
f = plt.figure(figsize=(20, 5))
plt.imshow(prediction_matrix, cmap='Paired', interpolation='nearest')
plt.xticks(range(n_classes), sign_names, rotation = 90)
plt.yticks(range(n_test), y_name)
plt.grid(which='minor')
plt.colorbar()
plt.show()

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

My network has very high confidence, ~1, on the first three predictions.

Although the network failed to determine the sign on the image of _Children Crossing_, the sign is in top-5 predictions. the network has ~0.2 confidence that the image is a _Children Crossing_.

Unfortunately, my network failed to determine the _Keep Left_ sign on the last image, the _Keep Left_ is not in any of top-5 prediction either. I also notice that the network think that the chance of the image is a _Keep Left_ sign is higher then a _Keep Right_ sign.

I think my network is not working weel on _Keep Left_ sign images. The number of _Keep Left_ sign images given in the raw dataset is only ~400. Perhaps I should add more images of _Keep Left_ signs in difference location and light condition to the raw training set, so that my network may have a better understanding of the sign.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.